In [19]:
import pandas as pd
import numpy as np
from data_science_projects import simple_neural_net as snn
from sklearn.model_selection import train_test_split

In [15]:
df = pd.read_csv('../../data/mushrooms.csv')

df_ring = df['ring-number']
df_ring.replace({'n':0, 'o':1, 't':2}, inplace=True)

df = df.drop(columns=['ring-number'])
df_onehot = pd.get_dummies(df, drop_first=True)
# I am creating new dataframes, since the data is not too big and it makes it easier to run with small changes later

df_onehot['ring-number'] = df_ring

cut_correlation = 0.3
# A cut of 0.3 was found to give the best scores for both cross-validation
# on the training set and the test set score for the decision tree.

corr = df_onehot.corr('pearson')
cut_df_onehot = df_onehot[corr['class_p'][corr['class_p'].abs()>cut_correlation].index]

cut_df_onehot.head()

,class_p,bruises_t,odor_f,odor_n,gill-spacing_w,gill-size_n,stalk-surface-above-ring_k,stalk-surface-above-ring_s,stalk-surface-below-ring_k,stalk-surface-below-ring_s,ring-type_l,ring-type_p,spore-print-color_h,spore-print-color_k,spore-print-color_n,spore-print-color_w,population_v,habitat_p
0,1,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0
1,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0
2,0,1,0,0,0,0,0,1,0,1,0,1,0,0,1,0,0,0
3,1,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0
4,0,0,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(cut_df_onehot.drop('class_p', 1), cut_df_onehot['class_p'], test_size = 0.1, random_state=10)

In [17]:
X_train.T.shape

(17, 7311)

In [39]:
layer_dims = [X_train.T.shape[0], X_train.T.shape[0], 1]

init_scale = 2.0 / np.sqrt(layer_dims)

parameters = snn.L_layer_model(np.array(X_train.T),
                               np.array(y_train).reshape([1,len(y_train)]),
                               layer_dims,
                               seed=1,
                               init_scale=init_scale,
                               learning_rate=0.01,
                               rel_tol=0.001,
                               hidden_active="relu",
                               outer_active="sigmoid",
                               cost_func="entropy",
                               print_cost = 100,
                               max_iter=10000,
                               batch_size=0)

Cost after iteration 0: 9.240752e-01
Cost after iteration 100: 5.762380e-02
Cost after iteration 200: 2.127717e-02
Cost after iteration 300: 8.200892e-03
Cost after iteration 400: 4.151216e-03
Cost after iteration 500: 2.476532e-03
Cost after iteration 600: 1.639977e-03
Cost after iteration 700: 1.164531e-03
Cost after iteration 800: 8.678548e-04
Cost after iteration 900: 6.701312e-04
Cost after iteration 1000: 5.324717e-04
Cost after iteration 1100: 4.322943e-04
Cost after iteration 1200: 3.576074e-04
Cost after iteration 1300: 3.001746e-04
Cost after iteration 1400: 2.550569e-04
Cost after iteration 1500: 2.190481e-04
Cost after iteration 1600: 1.897617e-04
Cost after iteration 1700: 1.656772e-04
Cost after iteration 1800: 1.456607e-04
Cost after iteration 1900: 1.288126e-04
Cost after iteration 2000: 1.145344e-04
Cost after iteration 2100: 1.023161e-04
Cost after iteration 2200: 9.167416e-05
Cost after iteration 2300: 8.202679e-05
Cost after iteration 2400: 7.378878e-05
Cost after i

In [93]:
y_pred, caches = snn.L_model_forward(np.array(X_test.T), parameters, hidden_active='relu', outer_active='sigmoid')

In [94]:
y_pred

array([[2.11129252e-05, 9.99912255e-01, 1.30035429e-14, 2.80303221e-07,
        9.99983865e-01, 9.99999991e-01, 4.94512349e-04, 1.88829814e-07,
        4.89591763e-15, 2.11129252e-05, 2.80303221e-07, 1.91533770e-07,
        2.80303221e-07, 2.80303221e-07, 9.99999995e-01, 1.00000000e+00,
        9.99999591e-01, 9.99992938e-01, 1.91533770e-07, 1.00000000e+00,
        7.02775450e-05, 1.60749645e-13, 1.64578216e-04, 4.89591763e-15,
        3.67424582e-05, 5.24185919e-09, 5.24185919e-09, 1.18402972e-05,
        9.99999678e-01, 5.24185919e-09, 9.99999995e-01, 4.89591763e-15,
        2.11129252e-05, 1.60749645e-13, 9.99983865e-01, 4.94512349e-04,
        4.94512349e-04, 2.11129252e-05, 5.51722420e-14, 2.80303221e-07,
        1.00000000e+00, 9.99999977e-01, 8.58138519e-09, 9.99969817e-01,
        2.28015921e-06, 5.94632171e-07, 5.49488064e-05, 7.02775450e-05,
        5.24185919e-09, 1.30035429e-14, 1.64578216e-04, 1.91533770e-07,
        5.24185919e-09, 9.99999991e-01, 9.99999977e-01, 2.803032

In [95]:
y_pred = np.where(y_pred<0.5, 0, 1)
y_pred

array([[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0,
        1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
        1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
        1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
        0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0,
        0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1,
        0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1,
        1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
        1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 

In [101]:
len(np.where(y_pred[0]==np.array(y_test))[0])/len(y_test)

1.0

In [97]:
np.array(y_test)[:10]

array([0, 1, 0, 0, 1, 1, 0, 0, 0, 0], dtype=uint8)

In [98]:
np.where(y_pred[0]==np.array(y_test))

(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
        104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
        117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
        130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
        143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
        156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
        169, 170, 171, 172, 173, 174, 175, 176, 177

In [100]:
y_pred[0][:10]

array([0, 1, 0, 0, 1, 1, 0, 0, 0, 0])